<a href="https://colab.research.google.com/github/alexlaurence/CLARA/blob/master/CLARA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CLARA (2019.0.2-alpha)

*Convolutional Learning for Awake Rodent-MRI signal Analysis*

[www.celestial.tokyo/~AlexLaurence/](http://www.celestial.tokyo/~AlexLaurence/)



This is an AI-driven tool for predicting noise in awake-rodent fMRI for data-cleaning pipelines in preclinical research.

* CLARA developed by [Alexander Laurence](mailto:alexander.adamlaurence@gmail.com) with the help of various open source neuroimaging tools (TensorFlow, Keras, nii2img, niwidgets).

* RatNet developed by [Alexander Laurence](mailto:alexander.adamlaurence@gmail.com), trained on n=24 male Sprague Dawley® Rats fMRI images (~30,000 images in total). 

## Introduction

### Relevance
Rodent MRI is a powerful model for preclinical research exploring physiology, pathology and drug development. Particularly, awake rodent functional MRI can allow us to assess concious changes in brain activity across different mental states. However, without anaesthesia, head movement and long interscan intervals can contribute to occasional signal dropout. As such, this can lead to image quality inconsistency as shown in the reduction of grey value intensity. By detecting the images with reduced intensity (i.e. dark images), we aim to repair potential dropouts that may confound the blood oxygen level dependent signals. The goal of this tool is to develop an objective method of defining and detecting signal dropout as dark slices in the effort to repair affected volumes. 

### Training Data
We scanned the brains of two groups of healthy male adult rats (n=24). The functional images were acquired by a 7-T scanner using a Fast Spin Echo sequence of a 64x64 matrix, TR=2500ms, ETL 16, and effective TE=36ms. 

Two experimenters inspected signal dropout at the coronal slices which could also be seen as dark bands in the axial and sagittal views. To provide an objective measurement, mean image intensity was calculated by using ImageJ. Subjective inspection from two experimenters shows highly consistent identification of dark images. Using the image intensity data, we plotted the distribution and established a threshold of detecting the dark images. We defined a threshold that best matching the subjective inspection and formed objective criteria that could be applied across datasets. 

### Objectives
CLARA provides rodent fMRI pre-clinical studies with a pipeline for automatically detecting signal dropout in the form of dark slices. Through this approach, overall consistency of image quality may improve translational validity for researchers.

# 0. Building the Model

![alt text](https://i.imgur.com/tAyfB72.png)


The model (RatNet) is built using Keras, utilizing TensorFlow as the backend. TensorFlow was chosen as the backend due to better performance over Theano, and the ability to visualize the neural network using TensorBoard.

For predicting two categories (good/bad), our model utilizes three convolutional layers, each having a depth of 32. A Max Pooling layer is applied after all three convolutional layers with size (2,2).

After pooling, the data is fed through a single fully connected layer of size 128, and finally to the output layer, consisting of 2 softmax nodes.

**Developers: Unfortunately I do not have permission from the lab to distribute the actual training data, or any raw files associated with it. Please use this as a guide towards building your own model for CLARA. As such RatNet remains closed-source. **

Let's begin by installing the Open source Computer Vision (OpenCV) library.

In [0]:
!pip install opencv-python

## 0.1 Shape Function
We can get the shape of multi-dimensional Tensor:

In [0]:
a = tf.truncated_normal([16,128,128,3])
sess = tf.Session()
sess.run(tf.initialize_all_variables())
sess.run(tf.shape(a))

Let's reshape this to a new 2D Tensor of shape[16  128*128*3]= [16 49152].

In [0]:
b=tf.reshape(a,[16,49152])
sess.run(tf.shape(b))

## 0.2 Reading Inputs

The inputs are 30,000 images that I used for my MSc research project. Typically, we divide our input data into 3 parts:

* **Training data**: we shall use 80% of the images for training.

* **Validation data**: 20% images will be used for validation. These images are taken out of training data to calculate accuracy independently during the training process.

* **Test set**: separate independent data for testing which has around 400 images. Sometimes due to something called Overfitting; after training, neural networks start working very well on the training data(and very similar images) i.e. the cost becomes very small, but they fail to work well for other images. For example, if you are training a classifier between dogs and cats and you get training data from someone who takes all images with white backgrounds. It’s possible that your network works very well on this validation data-set, but if you try to run it on an image with a cluttered background, it will most likely fail. So, that’s why we try to get our test-set from an independent source.

In [0]:
classes = ['good', 'bad']
num_classes = len(classes)
 
train_path='training_data'
 
# validation split
validation_size = 0.2
 
# batch size
batch_size = 16
 
data = dataset.read_train_sets(train_path, img_size, classes, validation_size=validation_size)

Now let's upload our training data as one zip file...

In [0]:
from google.colab import files
uploaded_training = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded_training[fn])))
  
if not os.path.exists("/content/" + train_path):
        os.makedirs("/content/" + train_path)
    
import shutil
shutil.move("/content/" + fn, "/content/" + train_path)

!unzip "/content/" + train_path + "/" + fn

The objective of our training is to learn the correct values of weights/biases for all the neurons in the network that work to do classification between 'good' image and 'bad' image. Let’s create functions to create initial weights quickly just by specifying the shape

In [0]:
def create_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

In [0]:
def create_biases(size):
    return tf.Variable(tf.constant(0.05, shape=[size]))

## 0.3 Creating Network Layers

**f.nn.conv2d** function can be used to build a convolutional layer which takes these inputs:

* **input**= the output(activation) from the previous layer. This should be a 4-D tensor. Typically, in the first convolutional layer, you pass n images of size width*height*num_channels, then this has the size [n width height num_channels]

* **filter**= trainable variables defining the filter. We start with a random normal distribution and learn these weights. It’s a 4D tensor whose specific shape is predefined as part of network design. If your filter is of size filter_size and input fed has num_input_channels and you have num_filters filters in your current layer, then filter will have following shape:



> `[filter_size filter_size num_input_channels num_filters]`



* **strides**= defines how much you move your filter when doing convolution. In this function, it needs to be a Tensor of size>=4 i.e. [batch_stride x_stride y_stride depth_stride]. batch_stride is always 1 as you don’t want to skip images in your batch. x_stride and y_stride are same mostly and the choice is part of network design and we shall use them as 1 in our example. depth_stride is always set as 1 as you don’t skip along the depth.

* **padding**=SAME means we shall 0 pad the input such a way that output x,y dimensions are same as that of input.

After convolution, we add the biases of that neuron, which are also learnable/trainable. Again we start with random normal distribution and learn these values during training.

Now, we apply max-pooling using tf.nn.max_pool function that has a very similar signature as that of conv2d function.

In [0]:
tf.nn.max_pool(value=layer, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

Finally, we use a RELU as our activation function which simply takes the output of max_pool and applies RELU using **tf.nn.relu**.

Here's our complete convolution layer:

In [0]:
def create_convolutional_layer(input,
               num_input_channels, 
               conv_filter_size,        
               num_filters):  
    
    ## We shall define the weights that will be trained using create_weights function.
    weights = create_weights(shape=[conv_filter_size, conv_filter_size, num_input_channels, num_filters])
    ## We create biases using the create_biases function. These are also trained.
    biases = create_biases(num_filters)
 
    ## Creating the convolutional layer
    layer = tf.nn.conv2d(input=input,
                     filter=weights,
                     strides=[1, 1, 1, 1],
                     padding='SAME')
 
    layer += biases
 
    ## We shall be using max-pooling.  
    layer = tf.nn.max_pool(value=layer,
                            ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1],
                            padding='SAME')
    ## Output of pooling is fed to Relu which is the activation function for us.
    layer = tf.nn.relu(layer)
 
    return layer

The Output of a convolutional layer is a multi-dimensional Tensor. We want to convert this into a one-dimensional tensor. This is done in the Flattening layer. We simply use the reshape operation to create a single dimensional tensor as defined below:

In [0]:
def create_flatten_layer(layer):
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:4].num_elements()
    layer = tf.reshape(layer, [-1, num_features])
 
    return layer

Now, let’s define a function to create a fully connected layer. We do this by adding a condition that allows the caller to add RELU to the layer.

In [0]:
def create_fc_layer(input,          
             num_inputs,    
             num_outputs,
             use_relu=True):
    
    #Let's define trainable weights and biases.
    weights = create_weights(shape=[num_inputs, num_outputs])
    biases = create_biases(num_outputs)
 
    layer = tf.matmul(input, weights) + biases
    if use_relu:
        layer = tf.nn.relu(layer)
 
    return layer

Now, let’s create a placeholder that will hold the input training images. All the input images are read in dataset.py file and resized to 128 x 128 x 3 size. Input placeholder x is created in the shape of [None, 128, 128, 3]. The first dimension being None means you can pass any number of images to it. For this program, we shall pass images in the batch of 16 i.e. shape will be [16 128 128 3]. Similarly, we create a placeholder y_true for storing the predictions. For each image, we have two outputs i.e. probabilities for each class. Hence y_pred is of the shape [None 2] (for batch size 16 it will be [16 2].

In [0]:
x = tf.placeholder(tf.float32, shape=[None, img_size,img_size,num_channels], name='x')
 
y_true = tf.placeholder(tf.float32, shape=[None, num_classes], name='y_true')
y_true_cls = tf.argmax(y_true, dimension=1)

We use the functions defined above to create various layers of the network.

In [0]:
layer_conv1 = create_convolutional_layer(input=x,
               num_input_channels=num_channels,
               conv_filter_size=filter_size_conv1,
               num_filters=num_filters_conv1)
 
layer_conv2 = create_convolutional_layer(input=layer_conv1,
               num_input_channels=num_filters_conv1,
               conv_filter_size=filter_size_conv2,
               num_filters=num_filters_conv2)
 
layer_conv3= create_convolutional_layer(input=layer_conv2,
               num_input_channels=num_filters_conv2,
               conv_filter_size=filter_size_conv3,
               num_filters=num_filters_conv3)
          
layer_flat = create_flatten_layer(layer_conv3)
 
layer_fc1 = create_fc_layer(input=layer_flat,
                     num_inputs=layer_flat.get_shape()[1:4].num_elements(),
                     num_outputs=fc_layer_size,
                     use_relu=True)
 
layer_fc2 = create_fc_layer(input=layer_fc1,
                     num_inputs=fc_layer_size,
                     num_outputs=num_classes,
                     use_relu=False)

## 0.4 Predictions

Now, let’s define the cost that will be minimized to reach the optimum value of weights. We will use a simple cost that will be calculated using a Tensorflow function softmax_cross_entropy_with_logits which takes the output of last fully connected layer and actual labels to calculate cross_entropy whose average will give us the cost.

In [0]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2, labels=y_true)
cost = tf.reduce_mean(cross_entropy)

## 0.5 Optimisation

Tensorflow implements most of the optimisation functions. We shall use AdamOptimizer for gradient calculation and weight optimization. We shall specify that we are trying to minimise cost with a learning rate of 0.0001.

In [0]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

In [0]:
batch_size = 16
 
x_batch, y_true_batch, _, cls_batch = data.train.next_batch(batch_size)
 
feed_dict_train = {x: x_batch, y_true: y_true_batch}
 
session.run(optimizer, feed_dict=feed_dict_tr)

where next_batch is a simple python function in dataset.py file that returns the next 16 images to be passed for training. Similarly, we pass the validation batch of images independently to in another session.run() call.

In [0]:
x_valid_batch, y_valid_batch, _, valid_cls_batch = data.valid.next_batch(train_batch_size)
 
feed_dict_val = {x: x_valid_batch, y_true: y_valid_batch}
 
val_loss = session.run(cost, feed_dict=feed_dict_val)

We can calculate the accuracy on validataion set using true labels(y_true) and predicted labels(y_pred).

In [0]:
correct_prediction = tf.equal(y_pred_cls, y_true_cls)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

We can calculate the validation accuracy by passing accuracy in session.run() and providing validation images in a feed_dict.

In [0]:
val_acc = session.run(accuracy,feed_dict=feed_dict_validate)

Similarly, we also report the accuracy for the training images.

In [0]:
acc = session.run(accuracy, feed_dict=feed_dict_train)

As, training images along with labels are used for training, so in general training accuracy will be higher than validation. We report training accuracy to know that we are at least moving in the right direction and are at least improving accuracy in the training dataset. After each Epoch, we report the accuracy numbers and save the model using saver object in Tensorflow.

In [0]:
saver.save(session, 'RatNet-model')

Here's the complete train function

In [0]:
def train(num_iteration):
    global total_iterations
    
    for i in range(total_iterations,
                   total_iterations + num_iteration):
 
        x_batch, y_true_batch, _, cls_batch = data.train.next_batch(batch_size)
        x_valid_batch, y_valid_batch, _, valid_cls_batch = data.valid.next_batch(batch_size)
 
        
        feed_dict_tr = {x: x_batch,
                           y_true: y_true_batch}
        feed_dict_val = {x: x_valid_batch,
                              y_true: y_valid_batch}
 
        session.run(optimizer, feed_dict=feed_dict_tr)
 
        if i % int(data.train.num_examples/batch_size) == 0: 
            val_loss = session.run(cost, feed_dict=feed_dict_val)
            epoch = int(i / int(data.train.num_examples/batch_size))    
            
            show_progress(epoch, feed_dict_tr, feed_dict_val, val_loss)
            saver.save(session, 'RatNet-model') 
 
 
    total_iterations += num_iteration

# 1. Input Data

## 1.1 Imports

In [0]:
!pip install niwidgets
!pip install nibabel nilearn
!pip install shutil

    100% |████████████████████████████████| 8.2MB 5.2MB/s 
    100% |████████████████████████████████| 2.3MB 12.5MB/s 
    100% |████████████████████████████████| 4.2MB 5.9MB/s 
    100% |████████████████████████████████| 532kB 22.8MB/s 
    100% |████████████████████████████████| 256kB 27.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/72/21/a0/023d49343789e831207a7e7050086a20ff6400e807b30282ac
  Stored in directory: /root/.cache/pip/wheels/7a/22/84/e51d1d5fff13256af8ba0f246ba2fd1196f631d32306ac2f63
Successfully built niwidgets ipywebrtc
    100% |████████████████████████████████| 2.3MB 13.5MB/s 
  Could not find a version that satisfies the requirement shutil (from versions: )
No matching distribution found for shutil


## 1.2 Select NIfTI file

Select a single NIfTI (.nii) image. Use the widget to verify your image. 

Nb. Widgets are not fully operational on Google Colab at the moment, but you can run these scripts elsewhere if needed.

In [0]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

#from niwidgets import NiftiWidget
#import nilearn.plotting as nip
#print('Plotting ' + fn)
#my_widget = NiftiWidget('/content/' + fn)
#my_widget.nifti_plotter(plotting_func=nip.plot_img, display_mode=['ortho', 'x', 'y', 'z'], colormap='gray')

Saving f105x1.nii to f105x1.nii
User uploaded file "f105x1.nii" with length 246112 bytes


In [0]:
from niwidgets import NiftiWidget
import nilearn.plotting as nip

for fn in uploaded.keys():
  my_widget = NiftiWidget('/content/' + fn)
  my_widget.nifti_plotter(plotting_func=nip.plot_img, display_mode=['ortho', 'x', 'y', 'z'], colormap='gray')

## 1.3 Convert NIfTI to Image 

Conversion tool developed from **nii2img** (https://github.com/medihack/nii2img).

Copyright © 2006-2013 Jessica Jesser and Kai Schlamp. All rights reserved. 

In [0]:
#@title Basic Header Information
slices = 15 #@param {type:"number"}
volumes = 1
totalimg = slices*volumes

### Imports

In [0]:
import os
import sys
import getopt
import nibabel as nib
import numpy
import scipy

import matplotlib.pylab as plt

import shutil

### Define Parameters

### Load Nii

Load NIfTI file according to parameters above

In [0]:
#@title Enter NIfTI Data Paramters

# params is the dictionary holding all needed parameters for the script
params = {
    'nii_filename': "",
    'start_slice': 0,
    'end_slice': None,
    'alternate_slice': 1,
    'plane': "ax",
    'output_prefix': "img",
    'format': "png",
    'verbose': False
}

first_slice = 0 #@param {type:"number"}
alt_slice = 1 #@param {type:"number"}
plane_view = "ax" #@param {type:"string"}
prefix_filename = "img" #@param {type:"string"}
file_format = "png" #@param {type:"string"}
output_folder = "/preprocessed/" #@param {type:"string"}

dst = "/content" + output_folder

if not os.path.exists(dst):
        os.makedirs(dst)

def load_nii(params):
    if not params['nii_filename']:
        print("Error! Missing NIFTI input file.")
        sys.exit(2)

    if params['verbose']:
        print("Loading NIfTI file")

    img = nib.load(params['nii_filename'])
    data = img.get_data()
    return data

for fn in uploaded.keys():
  data = load_nii(params ={
      'nii_filename': '/content/' + fn, 
      'start_slice': first_slice, 
      'end_slice': None, 
      'alternate_slice': alt_slice, 
      'plane': plane_view, 
      'output_prefix': prefix_filename, 
      'format': file_format, 
      'verbose': False
  })

### Normalise Nii

Normalize the color value range in the original nii image

In [0]:
def normalize_nii(params, data):
    if params['verbose']:
        print("Normalizing intensities")

normalize_nii(params, data)

### Extract Nii

Extract the images from the file you wish to continue with, alternation, start and end slice

In [0]:
def extract_img(params, data):
    if params['verbose']:
        print("Extracting images")
        
    for fn in uploaded.keys():
      print('Filename: ' + fn)

    image_counter = 0
    slice_counter = 0
    start_slice = params['start_slice'] or 0
    start_slice = start_slice - 1 if start_slice > 0 else start_slice
    end_slice = params['end_slice'] or data.shape[2]
    for i in range(start_slice, end_slice):
        if (slice_counter % params['alternate_slice']) == 0:
            if params['plane'] == "ax":
                slice = data[:, :, i]
            elif params['plane'] == "cor":
                slice = data[:, i, :]
            elif params['plane'] == "sag":
                slice = data[i, :, :]

            slice = numpy.rot90(slice)

            image_name = params['output_prefix'] + "-"+ fn[:-4] + "-" "{:0>2}".format(str(image_counter)) + "." + params['format']
            scipy.misc.imsave(image_name, slice)
            image_counter += 1
            if params['verbose']:
                sys.stdout.write(".")
                sys.stdout.flush()
            
        src = "/content/" + image_name
        shutil.move(src, dst)
        slice_counter += 1

    if params['verbose']:
        print("")
        
extract_img(params, data)

t = 0
plt.figure(figsize=(10, 50))
for n in range(totalimg):
  plt.subplot(18,5,n+1)
  plt.imshow(data[n])
  plt.title('%i' %t)
  t = t +1
  plt.axis('off')
#plt.suptitle("Awake Rodent MRI Slices")

Filename: f105x1.nii


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:217: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  im = toimage(arr, channel_axis=2)
/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:337: DeprecationWarning: `bytescale` is deprecated!
`bytescale` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
  cmin=cmin, cmax=cmax)


In [0]:
t = 0
plt.figure(figsize=(10, 50))

for n in range(totalimg):
  plt.subplot(18,5,n+1)
  plt.imshow(data[n])
  plt.title('%i' %t)
  t = t +1
  plt.axis('off')

## 1.4 Crop and Resize Image (WIP)

In [0]:
import sys
# from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from skimage import io
from skimage.transform import resize
import numpy as np

/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [0]:
#@title Crop/Resize Image
x_crop = 1 #@param {type:"number"}
y_crop = 1 #@param {type:"number"}
newsize = 109 #@param {type:"number"}

def create_directory(directory):
    '''
    Creates a new folder in the specified directory if the folder doesn't exist.
    INPUT
        directory: Folder to be created, called as "folder/".
    OUTPUT
        New folder in the current directory.
    '''
    if not os.path.exists(directory):
        os.makedirs(directory)


def crop_and_resize_images(path, new_path, cropx, cropy, img_size):
    '''
    Crops, resizes, and stores all images from a directory in a new directory.
    INPUT
        path: Path where the current, unscaled images are contained.
        new_path: Path to save the resized images.
        img_size: New size for the rescaled images.
    OUTPUT
        All images cropped, resized, and saved from the old folder to the new folder.
    '''
    create_directory(new_path)
    dirs = [l for l in os.listdir(path) if l != '.DS_Store']
    total = 0

    for item in dirs:
        img = io.imread(path+item)
        y,x = img.shape
        startx = x//2-(cropx//2)
        starty = y//2-(cropy//2)
        img = img[starty:starty+cropy,startx:startx+cropx]
        img = resize(img, (newsize,newsize))
        io.imsave(str(new_path + item), img)
        #src = "/content/preprocessed/" + item
        #dst = "/content/preprocessed/cr"
        #shutil.move(src, dst)
        total += 1
        print("Cropped/Resized: ", item, total)

In [0]:
crop_and_resize_images(path='/content/preprocessed/', new_path='/content/cropped/', cropx=x_crop, cropy=y_crop, img_size=newsize)

Cropped/Resized:  img-f105x1-02.png 1
Cropped/Resized:  img-f105x1-07.png 2
Cropped/Resized:  img-f105x1-10.png 3
Cropped/Resized:  img-f105x1-08.png 4
Cropped/Resized:  img-f105x1-14.png 5
Cropped/Resized:  img-f105x1-00.png 6
Cropped/Resized:  img-f105x1-09.png 7
Cropped/Resized:  img-f105x1-05.png 8
Cropped/Resized:  img-f105x1-06.png 9
Cropped/Resized:  img-f105x1-04.png 10
Cropped/Resized:  img-f105x1-13.png 11
Cropped/Resized:  img-f105x1-03.png 12
Cropped/Resized:  img-f105x1-11.png 13
Cropped/Resized:  img-f105x1-01.png 14
Cropped/Resized:  img-f105x1-12.png 15


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.6/dist-packages/skimage/io/_io.py:132: UserWarning: /content/preprocessed/img-f105x1-02.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/dist-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))
/usr/local/lib/python3.6/dist-packages/skimage/io/_io.py:132: UserWarning: /content/preprocessed/img-f105x1-07.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/dist-packages/skimage/io/_io.py:132: UserWarning: /content/preprocessed/img-f105x1-10.png is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/dist-packag

In [0]:
img = io.imread('/content/preprocessed/img-f105x1-00.png')
print(img.shape)

(100, 100)


## 1.5 Convert Image to Array (WIP)

Converts each image to an array, and appends each array to a new NumPy array, based on the image column equaling the image file name.

In [0]:
import time

import numpy as np
import pandas as pd
from PIL import Image


def change_image_name(df, column):
    """
    Appends the suffix '.jpeg' for all image names in the DataFrame
    INPUT
        df: Pandas DataFrame, including columns to be altered.
        column: The column that will be changed. Takes a string input.
    OUTPUT
        Pandas DataFrame, with a single column changed to include the
        aforementioned suffix.
    """
    return [i + '.jpeg' for i in df[column]]


def convert_images_to_arrays_train(file_path, df):
    """
    Converts each image to an array, and appends each array to a new NumPy
    array, based on the image column equaling the image file name.
    INPUT
        file_path: Specified file path for resized test and train images.
        df: Pandas DataFrame being used to assist file imports.
    OUTPUT
        NumPy array of image arrays.
    """

    lst_imgs = [l for l in df['train_image_name']]

    return np.array([np.array(Image.open(file_path + img)) for img in lst_imgs])


def save_to_array(arr_name, arr_object):
    """
    Saves data object as a NumPy file. Used for saving train and test arrays.
    INPUT
        arr_name: The name of the file you want to save.
            This input takes a directory string.
        arr_object: NumPy array of arrays. This object is saved as a NumPy file.
    OUTPUT
        NumPy array of image arrays
    """
    return np.save(arr_name, arr_object)

In [0]:
start_time = time.time()

labels = pd.read_csv("/content/trainLabels.csv")

print("Writing Train Array")
X_train = convert_images_to_arrays_train('/content/preprocessed/', labels)

print(X_train.shape)

print("Saving Train Array")
save_to_array('/content/X_train.npy', X_train)

print("--- %s seconds ---" % (time.time() - start_time))

# 2. Predict Noise

In [0]:
import tensorflow as tf
import numpy as np
import os,glob,cv2

Load a single image to try the model on.

File RatNet-model.meta contains the complete network graph and we can use this to recreate the graph later. We shall use a saver object provided by Tensorflow to do this.

In [0]:
saver = tf.train.import_meta_graph('RatNet-model.meta')

The file RatNet-model.data-00000-of-00001 contains the trained weights(values of variables) of the network. So, once we have recreated the graph, we shall restore the weights.

In [0]:
saver.restore(sess, tf.train.latest_checkpoint('./'))

In order to get the prediction of the network, we need to read & pre-process the input image in the same way(as training), get hold of y_pred on the graph and pass it the new image in a feed dict. So, let’s do that:

In [0]:
image = cv2.imread(filename)

# Resizing the image to our desired size and preprocessing will be done exactly as done during training
image = cv2.resize(image, (image_size, image_size), cv2.INTER_LINEAR)
images.append(image)
images = np.array(images, dtype=np.uint8)
images = images.astype('float32')
images = np.multiply(images, 1.0/255.0) 

#The input to the network is of shape [None image_size image_size num_channels]. Hence we reshape.
x_batch = images.reshape(1, image_size,image_size,num_channels)
 
graph = tf.get_default_graph()
 
y_pred = graph.get_tensor_by_name("y_pred:0")
 
## Let's feed the images to the input placeholders
x= graph.get_tensor_by_name("x:0") 
y_true = graph.get_tensor_by_name("y_true:0") 
y_test_images = np.zeros((1, 2)) 
 
feed_dict_testing = {x: x_batch, y_true: y_test_images}
result=sess.run(y_pred, feed_dict=feed_dict_testing)

Finally, we can run a new image of good/bad using predict script.

In [0]:
#@title Input Image Directory
# First, pass the path of the image
dir_path = os.path.dirname(os.path.realpath(__file__))
image_path="" #@param {type:"string"}
filename = dir_path +'/' +image_path
image_size=128
num_channels=3
images = []

# Reading the image using OpenCV
image = cv2.imread(filename)

# Resizing the image to our desired size and preprocessing will be done exactly as done during training
image = cv2.resize(image, (image_size, image_size),0,0, cv2.INTER_LINEAR)
images.append(image)
images = np.array(images, dtype=np.uint8)
images = images.astype('float32')
images = np.multiply(images, 1.0/255.0) 

#The input to the network is of shape [None image_size image_size num_channels]. Hence we reshape.
x_batch = images.reshape(1, image_size,image_size,num_channels)

## Let us restore the saved model 
sess = tf.Session()

# Step-1: Recreate the network graph. At this step only graph is created.
saver = tf.train.import_meta_graph('RatNet-model.meta')

# Step-2: Now let's load the weights saved using the restore method.
saver.restore(sess, tf.train.latest_checkpoint('./'))

# Accessing the default graph which we have restored
graph = tf.get_default_graph()

# Now, let's get hold of the op that we can be processed to get the output.
# In the original network y_pred is the tensor that is the prediction of the network
y_pred = graph.get_tensor_by_name("y_pred:0")

## Let's feed the images to the input placeholders
x= graph.get_tensor_by_name("x:0") 
y_true = graph.get_tensor_by_name("y_true:0") 
y_test_images = np.zeros((1, len(os.listdir('training_data')))) 


### Creating the feed_dict that is required to be fed to calculate y_pred 
feed_dict_testing = {x: x_batch, y_true: y_test_images}
result=sess.run(y_pred, feed_dict=feed_dict_testing)

# result is of this format [probabiliy_of_rose probability_of_sunflower]
print(result)

# 3. Future Work

* 4D time-series fMRI data support. Currenly only supports 3D (or 4D-segmented data).
* Program the neural network to retrain with new photos. This is a common practice, and only serves to optimize the model. Checks would be put in place to validate the images before being added to the classifier, in order to prevent low quality images from altering the classifier too drastically.

* Port the Keras model to CoreML, and deploy to an RatNet iOS application. CoreML is a framework designed by Apple for adding machine learning to iOS devices. This allows the ability of Python developers to export their models, convert the file to a .mlmodel file, and add the file to the iOS development cycle.

# 4. Acknowledgement

For my initial research in which this tool is based on, I acknowledge the Royal Society (grant) and the Wellcome Trust (ISSF) for funding support, as well as Dr. Cyril Pernet, Dr. Gerry Thompson, Dr. Maurits Jansen, and Dr. Anjanette Harris of the University of Edinburgh for technical advice. I also extend my gratitude to our colleagues at the Laboratory of Magnetic Resonance Research (Chang-Gung University, Taiwan), Sun-Lin Han, Dr. Jiun-Jie Wang, and Dr. Chih-Chien Tsai for collaborative research exchange that helped to build the foundations for the 2nd part of my MSc thesis. Finally, I would like to thank my supervisor Dr. Szu-Han Wang at the Centre of Clinical Brain Sciences (CCBS) who's unwavering patience, perseverance, and knowledge has supported me throughout my project.

In order to build CLARA, I made use of Ankit Sachan's helpful TensorFlow [tutorials](https://cv-tricks.com/tensorflow-tutorial/training-convolutional-neural-network-for-image-classification/) and [scripts](https://github.com/sankit1/cv-tricks.com/tree/master/Tensorflow-tutorials/tutorial-2-image-classifier) for image classification. I thank NIPY developers for [Neuro Widgets](https://github.com/nipy/niwidgets). I also made use of [nii2img](https://github.com/medihack/nii2img) for the conversion scripts, of which I thank Jessica Jesser and Kai Schlamp for developing. Furthermore, I also would like to extend my thanks to Siraj Raval for the inspiring deep learning in medical-imaging [guides](https://github.com/llSourcell/AI_in_Medicine_Clinical_Imaging_Classification#neural-network-architecture). 

Finally, I want to thank OpenCV and everyone at Google (especially at TensorFlow) for providing the tools to make all this possible.